In [4]:
import os

kddcup = os.path.join(os.getcwd(),'kddcup.data_10_percent.gz')

In [5]:
import pyspark 

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder.master("local").appName("myApp").config(conf=myConf).getOrCreate()

In [6]:
rdd_data = spark.sparkContext.textFile(kddcup)

In [9]:
rdd_data.take(1)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.']

In [11]:
rdd_data.take(1)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.']

In [35]:
from pyspark.sql import Row

temp_data = rdd_data.map(lambda x : x.split(','))
temp_data = temp_data.map(lambda p: Row(
                        duration=int(p[0]), 
                        protocol=p[1],
                        service=p[2],
                        flag=p[3],
                        src_bytes=int(p[4]),
                        dst_bytes=int(p[5]),
                        attack=p[41]
                    )
                )

In [36]:
df_data = spark.createDataFrame(temp_data)
df_data.show()

+--------+--------+-------+----+---------+---------+-------+
|duration|protocol|service|flag|src_bytes|dst_bytes| attack|
+--------+--------+-------+----+---------+---------+-------+
|       0|     tcp|   http|  SF|      181|     5450|normal.|
|       0|     tcp|   http|  SF|      239|      486|normal.|
|       0|     tcp|   http|  SF|      235|     1337|normal.|
|       0|     tcp|   http|  SF|      219|     1337|normal.|
|       0|     tcp|   http|  SF|      217|     2032|normal.|
|       0|     tcp|   http|  SF|      217|     2032|normal.|
|       0|     tcp|   http|  SF|      212|     1940|normal.|
|       0|     tcp|   http|  SF|      159|     4087|normal.|
|       0|     tcp|   http|  SF|      210|      151|normal.|
|       0|     tcp|   http|  SF|      212|      786|normal.|
|       0|     tcp|   http|  SF|      210|      624|normal.|
|       0|     tcp|   http|  SF|      177|     1985|normal.|
|       0|     tcp|   http|  SF|      222|      773|normal.|
|       0|     tcp|   ht

In [37]:
df_data.groupBy('attack').count().show()

+----------------+------+
|          attack| count|
+----------------+------+
|    warezmaster.|    20|
|          smurf.|280790|
|            pod.|   264|
|           imap.|    12|
|           nmap.|   231|
|   guess_passwd.|    53|
|        ipsweep.|  1247|
|      portsweep.|  1040|
|          satan.|  1589|
|           land.|    21|
|     loadmodule.|     9|
|      ftp_write.|     8|
|buffer_overflow.|    30|
|        rootkit.|    10|
|    warezclient.|  1020|
|       teardrop.|   979|
|           perl.|     3|
|            phf.|     4|
|       multihop.|     7|
|        neptune.|107201|
+----------------+------+
only showing top 20 rows



In [40]:
df_data.groupBy('attack').mean('duration').show()

+----------------+--------------------+
|          attack|       avg(duration)|
+----------------+--------------------+
|    warezmaster.|               15.05|
|          smurf.|                 0.0|
|            pod.|                 0.0|
|           imap.|                 6.0|
|           nmap.|                 0.0|
|   guess_passwd.|  2.7169811320754715|
|        ipsweep.|0.034482758620689655|
|      portsweep.|  1915.2990384615384|
|          satan.|0.040276903713027064|
|           land.|                 0.0|
|     loadmodule.|   36.22222222222222|
|      ftp_write.|              32.375|
|buffer_overflow.|                91.7|
|        rootkit.|               100.8|
|    warezclient.|   615.2578431372549|
|       teardrop.|                 0.0|
|           perl.|  41.333333333333336|
|            phf.|                 4.5|
|       multihop.|               184.0|
|        neptune.|                 0.0|
+----------------+--------------------+
only showing top 20 rows



In [47]:
from pyspark.sql import functions as F

df_data.groupBy('attack').pivot('protocol').agg(F.mean('src_bytes')).show()

+----------------+------------------+------------------+------------------+
|          attack|              icmp|               tcp|               udp|
+----------------+------------------+------------------+------------------+
|    warezmaster.|              null|              49.3|              null|
|          smurf.| 935.7722995833185|              null|              null|
|            pod.|1462.6515151515152|              null|              null|
|           nmap.|               8.0|               0.0|            189.88|
|           imap.|              null| 347.5833333333333|              null|
|   guess_passwd.|              null|125.33962264150944|              null|
|        ipsweep.|10.905464006938422|               0.0|              null|
|      portsweep.|               8.0| 667349.1106833494|              null|
|          satan.|25.666666666666668| 1.323446327683616|1.0235294117647058|
|           land.|              null|               0.0|              null|
|     loadmo

In [ ]:
()

In [48]:
df_data.groupBy('attack').pivot('protocol').mean('src_bytes').show()

+----------------+------------------+------------------+------------------+
|          attack|              icmp|               tcp|               udp|
+----------------+------------------+------------------+------------------+
|    warezmaster.|              null|              49.3|              null|
|          smurf.| 935.7722995833185|              null|              null|
|            pod.|1462.6515151515152|              null|              null|
|           nmap.|               8.0|               0.0|            189.88|
|           imap.|              null| 347.5833333333333|              null|
|   guess_passwd.|              null|125.33962264150944|              null|
|        ipsweep.|10.905464006938422|               0.0|              null|
|      portsweep.|               8.0| 667349.1106833494|              null|
|          satan.|25.666666666666668| 1.323446327683616|1.0235294117647058|
|           land.|              null|               0.0|              null|
|     loadmo

In [53]:
wiki_data

data/ds_bigdata_wiki.txt MapPartitionsRDD[134] at textFile at NativeMethodAccessorImpl.java:0

In [52]:
wiki_data = spark.sparkContext.textFile(os.path.join('data','ds_bigdata_wiki.txt'))

In [63]:
wiki_data.flatMap(lambda x : x.split()).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).map(lambda x : (x[1],x[0])).sortByKey(False).take(15)

[(21, '데이터'),
 (18, '데이터를'),
 (15, '및'),
 (14, '빅'),
 (12, '등'),
 (9, '있다.'),
 (8, '수'),
 (8, '데이터의'),
 (7, '미국'),
 (7, '통해'),
 (6, '유권자'),
 (6, '선거'),
 (6, '대한'),
 (6, '빅데이터'),
 (5, '활용한')]

In [65]:
wiki_data.flatMap(lambda x : x.split()).map(lambda x: (x,1)).groupByKey().mapValues(sum).map(lambda x: (x[1],x[0])).sortByKey(False).take(15)

[(21, '데이터'),
 (18, '데이터를'),
 (15, '및'),
 (14, '빅'),
 (12, '등'),
 (9, '있다.'),
 (8, '수'),
 (8, '데이터의'),
 (7, '미국'),
 (7, '통해'),
 (6, '유권자'),
 (6, '선거'),
 (6, '대한'),
 (6, '빅데이터'),
 (5, '활용한')]

In [66]:
_data = ["'김하나','English',100",
         "'김하나','Math',80",
         "'임하나','English',70",
         "'임하나','Math',100",
        "'김갑돌','English',82.3",
         "'김갑돌','Math',98.5"]

In [74]:
rdd_data = spark.sparkContext.parallelize(_data)
rdd_data.map(lambda x: x.split(',')).map(lambda x : (x[0],float(x[2]))).reduceByKey(lambda x,y : x+y).collect()

[("'김하나'", 180.0), ("'임하나'", 170.0), ("'김갑돌'", 180.8)]

In [75]:
rdd_data.map(lambda x: x.split(',')).map(lambda x : (x[1],float(x[2]))).reduceByKey(lambda x,y : x+y).collect()

[("'English'", 252.3), ("'Math'", 278.5)]

In [84]:
rdd_data.map(lambda x: x.split(',')).map(lambda x: (x[0], float(x[2]))).combineByKey(lambda value: (value,1),
                     lambda x,value: (x[0]+value, x[1]+1),                      
                     lambda x,y: (x[0]+y[0], x[1]+y[1])).collect()

[("'김하나'", (180.0, 2)), ("'임하나'", (170.0, 2)), ("'김갑돌'", (180.8, 2))]

In [87]:
avg = rdd_data.map(lambda x: x.split(',')).map(lambda x: (x[0], float(x[2]))).combineByKey(lambda value: (value,1),
                     lambda x,value: (x[0]+value, x[1]+1),                      
                     lambda x,y: (x[0]+y[0], x[1]+y[1]))

avg2 = avg.map(lambda x: (x[0],x[1][0]/x[1][1]))

for i, j in sorted(avg2.collect(),reverse=True):
    print(i,j)

'임하나' 85.0
'김하나' 90.0
'김갑돌' 90.4


In [89]:
r = requests.get('https://health.data.ny.gov/api/views/jxy9-yhdk/rows.json?accessType=DOWNLOAD')

wc = r.json()

In [110]:
wc['meta']['view']['columns'][1]

{'id': -1,
 'name': 'id',
 'dataTypeName': 'meta_data',
 'fieldName': ':id',
 'position': 0,
 'renderTypeName': 'meta_data',
 'format': {},
 'flags': ['hidden']}

# RDD 읽는 방법

In [ ]:
spark.sparkContext.parallelize(myList)
spark.sparkContext.textFile(os.path.join("data","ds_spark_wiki.txt"))


# DF 읽는 방법 

In [ ]:
spark.read.text(os.path.join("data", "ds_spark_wiki.txt"))

popDf = spark\
            .read.option("charset", "euc-kr")\
            .option("header", "true")\
            .csv(os.path.join("data","경기도 의정부시_인구현황_20200904.csv"))

df = spark\
        .read\
        .format('com.databricks.spark.csv')\
        .options(header='true', inferschema='true', delimiter=',')\
        .load(os.path.join('data','ds_spark.csv'))

df = spark\
        .read.options(header='true', inferschema='true', delimiter=',')\
        .csv(os.path.join('data', 'ds_spark.csv'))

# RDD to DF

In [112]:
rddDf=spark.createDataFrame(myRdd)

# row to df
from pyspark.sql import Row
_myRdd=myRdd.map(lambda x:Row(year=int(x[0]), name=x[1], height=int(x[2])))
_myDf=spark.createDataFrame(_myRdd)



#structType
from pyspark.sql.types import *
myRdd=spark.sparkContext.parallelize([(1, 'kim', 50.0), (2, 'lee', 60.0), (3, 'park', 70.0)])
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("height", DoubleType(), True)
])
_myDf = spark.createDataFrame(myRdd, schema)

NameError: name 'myRdd' is not defined

# json to spark_df

In [ ]:
jfile= os.path.join('src','ds_twitter_seoul_3.json')
tweetDf= spark.read.json(jfile)

In [ ]:
# url json spark_df

r=requests.get("https://raw.githubusercontent.com/jokecamp/FootballData/master/World%20Cups/all-world-cup-players.json")
wc=r.json()
_wcDf=spark.createDataFrame(wc)

# csv 읽는 방법 

In [ ]:
# 한글깨짐
spark.sparkContext.textFile(os.path.join("data","ds_spark_2cols.csv"))

# 한글안깨짐
popRddBin = spark.sparkContext.binaryFiles(os.path.join("../data","경기도 의정부시_인구현황_20210910.csv"))
_my = popRddBin.map(lambda x :x[1].decode('euc-kr'))



# filter


In [ ]:
stopwords = ['is','am','are','the','for','a', 'an', 'at']
myRdd_stop = myRdd2.flatMap(lambda x:x.split())\
                    .filter(lambda x: x not in stopwords)

for words in myRdd_stop.collect():
    print (words, end=' ')

# 불용어

In [ ]:
# txt 읽어서 불용어 및 단어빈도
import os

f=open(os.path.join("data", "ds_bigdata_wiki.txt"))

stopwords = set(['및','이를','등','이','이런','그와','또는','두', '이와', '전', '간'])
d = dict()
for sent in f.readlines():
    _words = sent.split()  # split into words
    for word in _words:
        if word not in stopwords: # remove stopwords
            if word not in d:
                d[word]=1
            else:
                d[word]=d[word]+1
# sorting
dSorted = {k: v for k, v in sorted(d.items(), key=lambda x: x[1], reverse=True)}

#빈도 5 이상
d1 = dict()
for key, value in dSorted.items():
    if value>5:
        d1[key]=value
        print (f"{key}\t{value}")

In [111]:
# rdd 불용어 + sorting

stopwords = set(['및','이를','등','이','이런','그와','또는','두', '이와', '전', '간'])

wc3=myRdd3\
    .flatMap(lambda x:x.split())\
    .filter(lambda x: x.lower() not in stopwords)\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(False)\
    .take(15)

print (type(wc3))
for i in wc3:
    print (i[0],i[1])

In [ ]:
# 공백 + , + .제거

wc = spark.sparkContext.textFile(os.path.join("data","ds_spark_wiki.txt"))\
    .flatMap(lambda x: x.split())\
    .map(lambda x: (x.lower().rstrip().lstrip().rstrip(',').rstrip('.'), 1))

wcReduceByKey = wc.reduceByKey(lambda x,y:x+y)
wcReduceByKey.sortByKey().take(10)

# datetime

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
toDate = udf(lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())

_wcDfCasted=wcDf.withColumn('date2', to_date(wcDf['DateOfBirth'], 'yyyy-MM-dd'))

# udf 조건

In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

height_udf = udf(lambda height: "taller" if height >=175 else "shorter", StringType())
heightDf=myDf.withColumn("height>175", height_udf(myDf.heightD))

# columnRenamed

In [ ]:
tDf=tDf.withColumnRenamed('id','ID')

# F함수

In [ ]:
from pyspark.sql import functions as F

myDf.agg(F.min(myDf.heightD),F.max(myDf.heightD),F.avg(myDf.heightD),F.sum(myDf.heightD)).show()
tDf.agg(F.min("height")).show()